In [2]:
from unsloth import FastLanguageModel
import torch

import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"


from huggingface_hub import snapshot_download


#model_path = snapshot_download("Llama-3.2-3B-Instruct-bnb-4bit", resume_download=True)
model_path = "./unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
# Then load it locally
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.659 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./llama-finetuned",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 10,
    num_train_epochs = 3,
    learning_rate = 2e-4,
    fp16 = True,
    optim = "adamw_8bit",
    save_strategy = "epoch",
    logging_steps = 10,
)


In [18]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="all_data - all_data.csv")["train"]
dataset = dataset.select_columns(["Content"])

from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    dataset_text_field="Content",
    max_seq_length=2048,
    packing=True,
    fp16=False,
    bf16= True)


Unsloth: Tokenizing ["Content"] (num_proc=16):   0%|          | 0/21184 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [20]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21,184 | Num Epochs = 3 | Total steps = 7,944
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 9,175,040/3,000,000,000 (0.31% trained)


Step,Training Loss
10,2.480700
20,2.553300
30,2.487900
40,2.538800
50,2.648200
60,2.714700
70,2.694800
80,2.699100
90,2.583700
100,2.630200


TrainOutput(global_step=7944, training_loss=2.582653803287556, metrics={'train_runtime': 13448.3382, 'train_samples_per_second': 4.726, 'train_steps_per_second': 0.591, 'total_flos': 4.178658881252598e+17, 'train_loss': 2.582653803287556})

In [7]:
model.save_pretrained("./fine")

In [8]:
tokenizer.save_pretrained("./fine")

('./fine/tokenizer_config.json',
 './fine/special_tokens_map.json',
 './fine/tokenizer.json')